# GPS Movie Creator

Adds a map with live GPS position according to the position of the camera used for the video.

### Data Gathering Process

You will need:
- GPS device or smartphone which is waterproof (or inside a waterproof bag).
- Waterproof camera (e.g. Go Pro)
- Kayak
- Camera attachment to Kayak. I've used an adjustable fishing rod holder with an extendable Go Pro pole to allow the Go Pro to be extended below the water. Whatever mount you use you should still tether the camera to the kayak in case it breaks.

Steps:
1.	Turn on the GPS device and wait for a good signal lock. If possible, set the GPS device to capture a fixed high sample rate (e.g. 1Hz).
2.	Start recording on your GPS device.
3.	Start the camera recording at a low frame rate with an acceptable resolution. I find 1080 resolution with 30fps worked well on my Go Pro.
4.	Film the time on your phone to the nearest second. This will allow you to synchronize the timing between the devices later.
5.	Position the camera so that it is submerged enough not to be obstructed by surface bubbles. Make sure the camera is pointing in the direction you want to film.
6.	Paddle over the region you want to survey. It is ideal if you have a GPS tracker with a live map display showing you where you have paddled. This allows you to make sure you cover the whole region.
7.	After finishing the paddle, turn the camera off and the GPS device off. The order of turning them off doesn't matter.
8.	Download the camera video file (Video file) and save in an accessible directory (vid_dir_name).
9.	Download the GPS file and convert it to a .gpx file. Save the .gpx file in this directory (GPS file). My GPS device produces a .fit file. To convert that to a .gpx file I uploaded it to my Strava account, then export as a .gpx file from Strava.
10.	Watch the Video file and pause the video when it shows the time on your phone or GPS device (from step 4). Note the time of the video when the device time changes to the next second. Subtract the time of the video from the device time to get the starting time of the video (start_time).
11.	Export a georeferenced image from mapping software which covers the whole region you surveyed. I used Nearmaps. For Nearmaps, choose the GDA2020 projection. Choose a resolution of 0.597m/pixel or lower. Save the .jpg file (Map image) and .jgw file from the zip file in this directory. If using other mapping software you just need to make sure you have the equivalent .jgw file values to match your map image.
12.	Follow remaining steps in this Jupyter Notebook.

### Software Inputs

Input files required:

- GPS file (.gpx)
- Video file (.mp4)
- Map image (.jpg)

Input parameters to specify
- Directory of Video file (vid_dir_name)
- GPS file name (gps_file_name)
- Video file name (vid_name)
- Map image name (map_name)

- Time that video starts to 1 second resolution (start_time)
- UTC time zone offset, e.g. Gold Coast is AEST which is 10 (UTC_offset)
- GDA2020 projection zone, e.g. Gold Coast is in zone 56 (mga_zone)

- .jgw file values from Map image. Open with a program such as notepad to read the values. (easting_scale, northing_scale, top_left_easting, top_left_northing)

In [1]:
import gpxpy
import gpxpy.gpx
import numpy as np
import datetime
from datetime import timedelta
import csv
import cv2
import os

# Input files - Change these file names to your file names
vid_dir_name = "F:/All_videos/Research_videos/GPS_movie" # I processed the videos from my external hardrive
gps_file_name = "Mooloolaba_GPS.gpx"
vid_name = "GPS_movie_Mooloolaba_example"
map_name = "EPSG7856_Date20220612_Lat-26.681206_Lon153.132295_Mpp0.299.jpg"

# Input parameters - Change these parameters to your parameters.
# format:     ss_mm_hh_DD_MM_YYYY
start_time = "21_40_09_25_06_2022"
UTC_offset = 10
mga_zone = 56
# values below are all from jgw file.
easting_scale = 0.298582
northing_scale = -0.298582
top_left_easting = 512922.633595
top_left_northing = 7049141.405968

""" ************** Processing *************** """
# other variables
GPS_video_height = 1280 # a larger value will take longer to process and require more free memory from your computer to process
GPS_video_max_frame_rate = 4 # a larger value will take longer to process

# get the datetime of the video  - year, month, day, hour, min, sec 
vid_time_start = datetime.datetime(int(start_time[15:19]), int(start_time[12:14]), 
                                   int(start_time[9:11]), int(start_time[6:8]), 
                                   int(start_time[3:5]), int(start_time[0:2])) - timedelta(hours=UTC_offset)
gpx_file = open(gps_file_name, 'r')
cap = cv2.VideoCapture(vid_dir_name + "/" + vid_name + ".MP4")
frame_rate = cap.get(cv2.CAP_PROP_FPS)
frames_total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# https://stackoverflow.com/questions/11105663/how-to-extract-gpx-data-with-python
# get all the gps points
lats = []
longs = []
times = []
gpx = gpxpy.parse(gpx_file)
for track in gpx.tracks: 
    for segment in track.segments: 
        for point in segment.points: 
            lats.append(point.latitude)
            longs.append(point.longitude)
            time_tmp = point.time
            times.append(time_tmp.replace(tzinfo=None))

# pair the closest gps point with each frame of the video, ignore gps points and frames which are further than 1s apart.
frame_indx_list = []
vid_time_list = []
lats_list = []
longs_list = []
frame_end_count = 0
while(cap.isOpened()):
    
    ret, frame = cap.read()
    if ret == True:

        vid_time = vid_time_start + timedelta(seconds= frame_end_count/frame_rate)
        
        # take datetime difference between all times
        first_pass = True
        for i in range(0, len(times), 1):
            if first_pass == True:
                if vid_time > times[i]:
                    lowest_diff_val = vid_time - times[i]
                else:
                    lowest_diff_val = times[i] - vid_time
                lowest_diff_indx = i
                first_pass = False
            else:
                if vid_time > times[i]:
                    diff_val = vid_time - times[i]
                else:
                    diff_val = times[i] - vid_time

                if diff_val < lowest_diff_val:
                    lowest_diff_val = diff_val
                    lowest_diff_indx = i

        closest_gpx_time = times[lowest_diff_indx]
        
        if abs(closest_gpx_time.timestamp() - vid_time.timestamp()) < 1.1:
            frame_indx_list.append(frame_end_count)
            vid_time_list.append(vid_time)
            lats_list.append(lats[lowest_diff_indx])
            longs_list.append(longs[lowest_diff_indx])
    
        frame_end_count = frame_end_count + 1    
        if frame_end_count % 500 == 0:
            print(frame_end_count)
    else:
        if frame_end_count >= frames_total:
            break
        else:
            print("dodgy frame skipped")
            frame_end_count = frame_end_count + 1
            if frame_end_count % 500 == 0:
                print(frame_end_count)

# create a gps csv file for converting to GDA2020
with open(vid_name + ".csv", "wb") as csv_file:
    for i in range(0, len(lats_list), 1):
        cell_line = np.array([[i, lats_list[i], longs_list[i], mga_zone]])
        np.savetxt(csv_file, cell_line, delimiter=",", fmt=['%i', '%f', '%f', '%i'], newline="\n")

print("done")

500
1000
1500
2000
2500
3000
3500
4000
4500
done


Process the csv file created from the above cell through this website. The csv file should be the vid_name with .csv at the end.

https://geodesyapps.ga.gov.au/grid-coordinate-batch-processing - works well in Chrome browser

Save the created file in this directory with the same filename as the original with "_movie" at the end.

Now run the cell below to create the GPS movies. Note that the videos are saved in parts so that the program doesn't crash from using too much memory. If you want to put all the movies into 1 movie, just put them together using video editing software such as OpenShot Video Editor.

The below cell takes about 3-5 minutes to run per minute of the video being processed.

In [ ]:
x_vals_map = []
y_vals_map = []
# read in GDA coordinates and convert to pixel coordinates on the map image
with open(vid_name + "_movie" + ".csv") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count < 1:
            # not a data cell
            pass
        # if at the end of a wave track
        else:
            x_vals_map.append(int((float(row[6]) - top_left_easting)/easting_scale))
            y_vals_map.append(int((float(row[7]) - top_left_northing)/northing_scale))
        
        line_count = line_count + 1

# get map image
map_image = cv2.imread(map_name)

# first plot all the points with colour scaling over time
for i in range(0, len(x_vals_map), 1):
    cv2.circle(map_image, (int(x_vals_map[i]), int(y_vals_map[i])), radius=3, thickness=-1, 
               color=[0, int(255 - 255*(i/len(x_vals_map))), int(255*(i/len(x_vals_map)))])

if GPS_video_max_frame_rate >= frame_rate:
    frame_skip_mod = 1
    rate_print = frame_rate
elif GPS_video_max_frame_rate < frame_rate:
    frame_skip_mod = int(frame_rate/GPS_video_max_frame_rate)
    rate_print = GPS_video_max_frame_rate
    
# now create a list of frames connected to the map, with a circle plotted on each map location
mov_img_list = []
vid_part_count = 1
for i in range(0, len(frame_indx_list), 1):
    
    if i % frame_skip_mod == 0:
        pass
    else:
        continue
    
    map_ann_image = map_image.copy()
    cv2.circle(map_ann_image, (int(x_vals_map[i]), int(y_vals_map[i])), radius=8, thickness=-1, color=[235, 219, 52])
    map_ann_image = cv2.resize(map_ann_image, 
                               (int(GPS_video_height*(map_ann_image.shape[1]/map_ann_image.shape[0])), GPS_video_height))
    
    cap.set(1, frame_indx_list[i])
    ret, frame_image = cap.read()
    if ret == False:
        continue
    frame_image = cv2.resize(frame_image, 
                       (int(GPS_video_height*(frame_image.shape[1]/frame_image.shape[0])), GPS_video_height))
    
    date_text = str(vid_time_list[i] + timedelta(hours=UTC_offset))
    cv2.putText(frame_image, date_text[0:19], (int(frame_image.shape[1]*0.1), int(frame_image.shape[0]*0.1)),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
    comb_img = np.concatenate((frame_image, map_ann_image), axis=1)
    mov_img_list.append(comb_img)
    
    # print loop index every 10 seconds processed
    if i % int(frame_skip_mod*4*10) == 0:
        print(i)
    # save video at every minute
    if i % int(frame_skip_mod*4*60) == 0 and i > 0:
        # create video
        video_recorded = cv2.VideoWriter(vid_dir_name + "/" + vid_name + "_GPS_movie_part_" + str(vid_part_count) + ".mp4",
                                         cv2.VideoWriter_fourcc(*'DIVX'), rate_print,
                                         (mov_img_list[0].shape[1], mov_img_list[0].shape[0]))
        print("writing a video")
        for i in range(0, len(mov_img_list), 1):
            video_recorded.write(mov_img_list[i])
        video_recorded.release()
        vid_part_count = vid_part_count + 1
        mov_img_list = []

# create video
video_recorded = cv2.VideoWriter(vid_dir_name + "/" + vid_name + "_GPS_movie_part_" + str(vid_part_count) + ".mp4",
                                 cv2.VideoWriter_fourcc(*'DIVX'), rate_print,
                                 (mov_img_list[0].shape[1], mov_img_list[0].shape[0]))
print("writing a video")
for i in range(0, len(mov_img_list), 1):
    video_recorded.write(mov_img_list[i])
video_recorded.release()

print("done")

0
280
560
840
1120
1400
1680
writing a video
1960
2240
2520
